<a href="https://colab.research.google.com/github/snpushpi/Covid19_Contact_tracing_repo_beta/blob/master/data_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code has been used for processing data from microsoft geolife trajectories :)

In [ ]:

import read_geolife
import pandas as pd
import numpy as np
import pptk
def make_list(frame):
    mod_df0 = frame.loc[:,'modified_lat':'modified_lon']
    new_list0lat = mod_df0['modified_lat'].to_list()
    new_list0lon = mod_df0['modified_lon'].to_list()
    l =[]
    length = len(new_list0lat)
    for i in range(length):
        l.append((new_list0lat[i],new_list0lon[i]))
    return l
def sublists(list1, list2):
    subs = []
    for i in range(len(list1)-1):
        for j in range(len(list2)-1):
            if list1[i]==list2[j] and list1[i+1]==list2[j+1]:
                m = i+2
                n = j+2
                while m<len(list1) and n<len(list2) and list1[m]==list2[n]:
                    m += 1
                    n += 1
                subs.append((i,m,j,n,list1[i:m]))
    return subs
def ordered_intersection(list1, list2):
    subls = sublists(list1, list2)
    if len(subls)==0:
        return []
    else:
        max_len = max(len(subl[4]) for subl in subls)
        return [subl for subl in subls if len(subl[4])==max_len]
def ordered_intersection1(list1, list2):
    set_2 = frozenset(list2)
    intersection = [x for x in list1 if x in set_2]
    return intersection
def modifier_lat(a):
    return float(int(a)+float(str(a)[2:7]))
def modifier_lon(a):
    return float(int(a)+float(str(a)[3:8]))
#df = read_geolife.read_all_users('Data')
#df.to_pickle('geolife.pkl')
df = pd.read_pickle('geolife.pkl')
#p = np.c_[df['lon'], df['lat'], np.zeros(len(df))]
#v = pptk.viewer(p)
#v.attributes(df['alt'])
#print(df[1000:1020])
#print(df[str(df.lat)[3:7]=='9999'])
df['modified_lat']=df.lat.apply(modifier_lat)
df['modified_lon']=df.lon.apply(modifier_lon)
print(len(df))
new_df = []
for i in range(15):
    new_df.append(df[df.user==i])
list_of_coors = []
for elt in new_df:
    list_of_coors.append(make_list(elt))
big_dict = {}
length = len(list_of_coors)
#print(check_dict,len(list_of_coors[0]), len(list_of_coors[2]))
#print(ordered_intersection(list_of_coors[0],list_of_coors[2]))
new_check_list = [list_of_coors[i][:2000] for i in range(length)]
for i in range(length):
    for j in range(i+1,length):
        inter = ordered_intersection(new_check_list[i], new_check_list[j])
        if len(inter)!=0:
            big_dict[(i,j)]=inter[0]
check_dict={e:len(big_dict[e][4]) for e in big_dict}
#print(check_dict)
#print(new_check_list[3][990:1240], new_check_list[4][573:823])
querier_list = new_check_list[3][990:1240]
infected_list1 = new_check_list[4][573:823]
infected_list2 = new_check_list[4][323:573]
infected_list3 = new_check_list[4][823:1073]
print(ordered_intersection(querier_list, infected_list1))
print(ordered_intersection(querier_list, infected_list2))
print(ordered_intersection(querier_list, infected_list3))